<a href="https://colab.research.google.com/github/fsaantibanez/datapath/blob/main/practica_pyspark_proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EjemploSpark").getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

from pyspark.sql.functions import lit
from pyspark.sql.functions import avg, round
from pyspark.sql.functions import sum
from pyspark.sql.functions import min, max
from pyspark.sql.functions import when

from pyspark.sql.functions import concat_ws

In [3]:
schema = StructType([
    StructField("Nombre", StringType(), True),
    StructField("Edad", IntegerType(), True),
    StructField("Ciudad", StringType(), True)
])

data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

df = spark.createDataFrame(data, schema=schema)

In [4]:
# 2. Mostrar solo los nombres de las personas del DataFrame.
df.select("Nombre").show()

+-------+
| Nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



In [5]:
# 3. Filtrar personas cuya edad sea mayor o igual a 25.
df.filter(df.Edad >= 25).show()

+------+----+-----------+
|Nombre|Edad|     Ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



In [6]:
# 4. Agregar una nueva columna "Pais" con un valor constante para todas las filas.
df = df.withColumn("Pais", lit("EEUU"))
df.show()

+-------+----+-----------+----+
| Nombre|Edad|     Ciudad|Pais|
+-------+----+-----------+----+
|  Alice|  25|   New York|EEUU|
|    Bob|  30|Los Angeles|EEUU|
|Charlie|  22|    Chicago|EEUU|
+-------+----+-----------+----+



In [7]:
# 5. Calcular el promedio de edad de todas las personas.
df.select(round(avg("Edad"),1)).show()

+-------------------+
|round(avg(Edad), 1)|
+-------------------+
|               25.7|
+-------------------+



In [8]:
# 6. Ordenar el DataFrame por edad en orden descendente.
df.orderBy(df.Edad.desc()).show()

+-------+----+-----------+----+
| Nombre|Edad|     Ciudad|Pais|
+-------+----+-----------+----+
|    Bob|  30|Los Angeles|EEUU|
|  Alice|  25|   New York|EEUU|
|Charlie|  22|    Chicago|EEUU|
+-------+----+-----------+----+



In [9]:
# 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad.
df.groupby("Ciudad").count().show()

+-----------+-----+
|     Ciudad|count|
+-----------+-----+
|   New York|    1|
|Los Angeles|    1|
|    Chicago|    1|
+-----------+-----+



In [10]:
# 8. Renombrar la columna "Nombre" a "NombreCompleto".
df = df.withColumnRenamed("Nombre","NombreCompleto")
df.show()

+--------------+----+-----------+----+
|NombreCompleto|Edad|     Ciudad|Pais|
+--------------+----+-----------+----+
|         Alice|  25|   New York|EEUU|
|           Bob|  30|Los Angeles|EEUU|
|       Charlie|  22|    Chicago|EEUU|
+--------------+----+-----------+----+



In [11]:
# 9. Eliminar la columna "Edad" del DataFrame.
df.drop('Edad').show()

+--------------+-----------+----+
|NombreCompleto|     Ciudad|Pais|
+--------------+-----------+----+
|         Alice|   New York|EEUU|
|           Bob|Los Angeles|EEUU|
|       Charlie|    Chicago|EEUU|
+--------------+-----------+----+



In [12]:
# 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años.
df.createOrReplaceTempView("personas")
result = spark.sql("select * FROM personas WHERE Edad > 20")
result.show()

+--------------+----+-----------+----+
|NombreCompleto|Edad|     Ciudad|Pais|
+--------------+----+-----------+----+
|         Alice|  25|   New York|EEUU|
|           Bob|  30|Los Angeles|EEUU|
|       Charlie|  22|    Chicago|EEUU|
+--------------+----+-----------+----+



In [13]:
# 11. Calcular la suma total de todas las edades.
df.select(sum("Edad")).show()

+---------+
|sum(Edad)|
+---------+
|       77|
+---------+



In [14]:
# 12. Calcular la edad mínima y máxima de todas las personas.
df.select(min("Edad"),max("Edad")).show()

+---------+---------+
|min(Edad)|max(Edad)|
+---------+---------+
|       22|       30|
+---------+---------+



In [15]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30.
df.filter((df.Ciudad == 'Chicago') &(df.Edad < 30 )).show()

+--------------+----+-------+----+
|NombreCompleto|Edad| Ciudad|Pais|
+--------------+----+-------+----+
|       Charlie|  22|Chicago|EEUU|
+--------------+----+-------+----+



In [16]:
# 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad.
df = df.withColumn("EdadDuplicada",df.Edad*2)
df.show()

+--------------+----+-----------+----+-------------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|
+--------------+----+-----------+----+-------------+
|         Alice|  25|   New York|EEUU|           50|
|           Bob|  30|Los Angeles|EEUU|           60|
|       Charlie|  22|    Chicago|EEUU|           44|
+--------------+----+-----------+----+-------------+



In [17]:
# 15. Convertir todas las edades en años a meses.
df = df.withColumn("EdadenMeses",df.Edad*12)
df.show()

+--------------+----+-----------+----+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadenMeses|
+--------------+----+-----------+----+-------------+-----------+
|         Alice|  25|   New York|EEUU|           50|        300|
|           Bob|  30|Los Angeles|EEUU|           60|        360|
|       Charlie|  22|    Chicago|EEUU|           44|        264|
+--------------+----+-----------+----+-------------+-----------+



In [18]:
# 16. Contar el número total de personas en el DataFrame.
print(f"Hay {df.count()} personas en la base de datos")

Hay 3 personas en la base de datos


In [19]:
# 17. Filtrar personas cuya edad sea un número par.
df.filter(df.Edad%2 == 0).show()

+--------------+----+-----------+----+-------------+-----------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadenMeses|
+--------------+----+-----------+----+-------------+-----------+
|           Bob|  30|Los Angeles|EEUU|           60|        360|
|       Charlie|  22|    Chicago|EEUU|           44|        264|
+--------------+----+-----------+----+-------------+-----------+



In [20]:
# 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+).
df.withColumn("RangoEdad", when((df.Edad >=0)& (df.Edad <=20),"0-20").
              when((df.Edad >=21)& (df.Edad <=40),"21-40").
              when((df.Edad >=41)& (df.Edad <=60),"41-60").
              otherwise("61+")).groupBy("RangoEdad").count().show()

+---------+-----+
|RangoEdad|count|
+---------+-----+
|    21-40|    3|
+---------+-----+



In [21]:
# 19. Contar cuántas personas tienen el mismo nombre.
df.groupBy("NombreCompleto").count().show()

+--------------+-----+
|NombreCompleto|count|
+--------------+-----+
|         Alice|    1|
|       Charlie|    1|
|           Bob|    1|
+--------------+-----+



In [22]:
# 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal".
df = df.withColumn("InformacionPersonals",concat_ws(", ",df.NombreCompleto,df.Ciudad))
df.show()

+--------------+----+-----------+----+-------------+-----------+--------------------+
|NombreCompleto|Edad|     Ciudad|Pais|EdadDuplicada|EdadenMeses|InformacionPersonals|
+--------------+----+-----------+----+-------------+-----------+--------------------+
|         Alice|  25|   New York|EEUU|           50|        300|     Alice, New York|
|           Bob|  30|Los Angeles|EEUU|           60|        360|    Bob, Los Angeles|
|       Charlie|  22|    Chicago|EEUU|           44|        264|    Charlie, Chicago|
+--------------+----+-----------+----+-------------+-----------+--------------------+

